In [1]:
import sys
import pyzed.sl as sl
import cv2

import math
import numpy as np
import random
from matplotlib import pyplot as plt

import get_clear_hand_mask
import find_fingertip
import tracking


In [2]:
# def Dis_pt2plane(pts, a, b, c, d):
#     """
#     Compute the distance from points to the plane
#     """
#     normal = math.sqrt(a*a+b*b+c*c)
#     if normal == 0:
#         normal = 1
    
#     v = np.array([a,b,c])
#     dis = abs(np.dot(pts,v.T)+d)/normal
#     return dis

# def get_Plane(sampts):
#     """
#     Compute the equation of the plane
#     """
#     p1 = sampts[0]
#     p2 = sampts[1]
#     p3 = sampts[2]
    
#     a = ( (p2[1]-p1[1])*(p3[2]-p1[2])-(p2[2]-p1[2])*(p3[1]-p1[1]) )
#     b = ( (p2[2]-p1[2])*(p3[0]-p1[0])-(p2[0]-p1[0])*(p3[2]-p1[2]) )
#     c = ( (p2[0]-p1[0])*(p3[1]-p1[1])-(p2[1]-p1[1])*(p3[0]-p1[0]) )
#     d = ( 0-(a*p1[0]+b*p1[1]+c*p1[2]) )
    
#     return a,b,c,d

# def Random3points(points3D, ConfidenceIndex):
#     """
#     Random choose 3 Confidence points
#     """
#     sample_number = 3
#     sample_point_index = random.sample(range(ConfidenceIndex.shape[0]), sample_number)
#     sample_points = np.zeros((sample_number,3))
#     for i in range(sample_number):
#         Confidence_point_index = sample_point_index[i]
#         index = ConfidenceIndex[Confidence_point_index]
#         y = index[0]
#         x = index[1]
#         sample_points[i] = points3D[y][x]
#     return sample_points

# # def Random3points(points3D):
# #     sample_number = 3
# #     sample_point_index = random.sample(range(points3D.shape[0]*points3D.shape[1]), sample_number)
# #     sample_points = np.zeros((sample_number,3))
# #     for i in range(sample_number):
# #         index = sample_point_index[i]
# #         y = index // points3D.shape[1]
# #         x = index % points3D.shape[1]
# #         sample_points[i] = points3D[y][x]
# #     return sample_points

# def get_inliner_num(points3D,a,b,c,d,inliner_threshold):
#     """
#     Compute the liner points which distance to plane < threshold
#     Also get distance from points to the plane (new Depth Image which re-project depth pixels in surface plane)
#     """
#     inliner_num = 0
    
#     dist = Dis_pt2plane(points3D,a,b,c,d)
#     inliner_mask = dist < inliner_threshold
#     inliner_num = np.sum(inliner_mask)
#     return inliner_num, inliner_mask, dist

# def RANSAM(points3D, ConfidenceIndex, ransac_iteration = 500, inliner_threshold = 10):
#     best_inlinernum = -1
#     best_inlinernum = 0
#     best_plane = np.zeros((1,4))
#     best_depthImage = np.zeros((points3D.shape[0],points3D.shape[1]))
#     best_plane_mask = np.zeros((points3D.shape[0],points3D.shape[1]))
# #     best_sampts = np.zeros((3,3))
    
# #     print(points3D.shape,points3D[80:90,110])
#     for i in range(ransac_iteration):
#         sampts = Random3points(points3D, ConfidenceIndex)
#         a,b,c,d = get_Plane(sampts)
        
#         inliner_num, inliner_mask, depthImage = get_inliner_num(points3D,a,b,c,d,inliner_threshold)
#         if(inliner_num > best_inlinernum):
#             best_inlinernum = inliner_num
#             best_plane = np.array([a,b,c,d])
#             best_plane_mask = inliner_mask
#             best_depthImage = depthImage
# #             best_sampts = sampts

#         if (best_inlinernum > 480000):
#             print("Inliner Number\n", best_inlinernum)
#             print("Inliner plane\n", best_plane)
#             return best_plane, best_depthImage, best_plane_mask
            
            
#     print("Inliner Number\n", best_inlinernum)
#     print("Inliner plane\n", best_plane)
#     return best_plane, best_depthImage, best_plane_mask

In [3]:
def candy_edge_with_skin_detect(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb) # 把圖像轉換到YUV色域
    (y, cr, cb) = cv2.split(ycrcb) # 圖像分割, 分別獲取y, cr, br通道圖像
    # 高斯濾波, cr 是待濾波的源圖像數據, (5,5)是值窗口大小, 0 是指根據窗口大小來計算高斯函數標準差
    kernel_size = 5
    cr1 = cv2.GaussianBlur(cr, (kernel_size, kernel_size), 0) # 對cr通道分量進行高斯濾波
    # 根據OTSU算法求圖像閾值, 對圖像進行二值化_, 
    _ ,skin1 = cv2.threshold(cr1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    
    cv2.imshow("Skin Cr+OSTU", skin1 )
    return skin1

In [4]:
def main():

    init = sl.InitParameters()
    init.camera_resolution = sl.RESOLUTION.RESOLUTION_HD720
    init.depth_mode = sl.DEPTH_MODE.DEPTH_MODE_PERFORMANCE

    #read from streaming
    sys.argv[1] = '127.0.0.1'
    if (len(sys.argv) > 1) :
        ip = sys.argv[1]
        init.set_from_stream(ip)
    else :
        print('Usage : python3 streaming_receiver.py ip')
        exit(1)
        
    #read from SVO file
#     filepath = 'record_1028_6.svo'
#     print("Reading SVO file: {0}".format(filepath))

#     init = sl.InitParameters(svo_input_filename=filepath,svo_real_time_mode=False)
    
    init.depth_minimum_distance = 150 # Set the minimum depth perception distance to 15cm
#     print(init.depth_mode)#PERFORMANCE

    cam = sl.Camera()
    status = cam.open(init)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit(1)

    runtime = sl.RuntimeParameters()
    runtime.sensing_mode = sl.SENSING_MODE.SENSING_MODE_FILL 
    
    image = sl.Mat() #image
    depth = sl.Mat() #depth map
    depth_for_display = sl.Mat() #depth map,scale its values to [0, 255]
    point_cloud = sl.Mat() #colored point cloud.
    confidence_map = sl.Mat() # confidence_map  [0 ,100], 0 :the best confidence value; 100 :the most unconfident one.

    key = ''
    First = True
    print("  Quit : CTRL+C\n")
    while key != 113:
        err = cam.grab(runtime)
        if (err == sl.ERROR_CODE.SUCCESS) :
            # Retrieve left image
            cam.retrieve_image(image, sl.VIEW.VIEW_LEFT)
            cam.retrieve_image(depth_for_display, sl.VIEW.VIEW_DEPTH)
            
            # Load Image data into a numpy array
            RGB_image = image.get_data()
            gray_image = cv2.cvtColor(RGB_image, cv2.COLOR_BGR2GRAY)
            
            
            # Retrieve depth map. Depth is aligned on the left image
            cam.retrieve_measure(depth, sl.MEASURE.MEASURE_DEPTH)
            # Load depth data into a numpy array
            depth_data = depth.get_data()
            
            # Retrieve colored point cloud. Point cloud is aligned on the left image.
            cam.retrieve_measure(point_cloud, sl.MEASURE.MEASURE_XYZRGBA)
            # Load point cloud into a numpy array
            points3D_color = point_cloud.get_data()
            points3D = points3D_color[:,:,:3] # only need x,y,z 
            points3D = np.where(points3D == np.nan, 1111, points3D)# replace nan to 1111111
            
            
            #get confidence index : index which value is not NAN 
            Confidence_index = np.argwhere(~np.isnan(depth_data[200:520,300:980]))#center region of image
#             Confidence_index = np.argwhere(~np.isnan(depth_data[:520,300:980]))
            
#             # Retrieve confidence_map.
#             cam.retrieve_measure(confidence_map, sl.MEASURE.MEASURE_CONFIDENCE)
#             # Load confidence_map into a numpy array
#             confidence_img = confidence_map.get_data()
#             confidence_index = np.argwhere(confidence_img <= 60)

            
            #####
            if First == True:
                #RANSAM to get surface plane
                surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 100, inliner_threshold = 10)
                First = False
                print("firstly RANSAM")
            else:
                #Ransam every times ???
#                 Confidence_index = np.argwhere(plane_mask == True)
#                 surface_plane, depthImg, plane_mask = get_clear_hand_mask.RANSAM(points3D, Confidence_index, ransac_iteration = 10, inliner_threshold = 10)

                #
                depthImg = get_clear_hand_mask.get_depth_map(points3D, surface_plane)
                #Canny edge map(infrared image) + threshold based edge map(depth image)
                Cannyedges, Threshold_based_edge, Edge_map, grayImage = get_clear_hand_mask.get_edge_map(gray_image, depthImg)
#                 Test skin_detect than Canny edge
                skinImg = candy_edge_with_skin_detect(RGB_image)

#                 #Get hight region by Hight and record its position
                High_region_Image, high_region_list = get_clear_hand_mask.get_high_region(depthImg, plane_mask)
#                 #Get Hand mask by Flood fill from high region position with Edge map and check wheather Flood_fill_with_edge_reasonable
                Hand_mask_Image = get_clear_hand_mask.get_reasonable_Hand_mask(Edge_map, high_region_list, High_region_Image, points3D)
#                 #find fingertip
                cnt, contours_image, hand_center, fingertips = find_fingertip.find_fingertip(Hand_mask_Image)#Hand_mask_Image skinImg
#                 # tracking user hand and fingertips
                hands_hand, fingertips_pos, tracking_image = tracking.hand_tracking(hand_center, fingertips, points3D, contours_image)

            
            #####
                #Check result
                cv2.imshow("Depth", depthImg.astype(np.uint8))
#                 cv2.imshow("plane_mask", plane_mask.astype(np.uint8)*255)
#                 cv2.imshow("Cannyedges", Cannyedges)
                cv2.imshow("Threshold_based_edge", Threshold_based_edge.astype(np.uint8)*255)
                cv2.imshow("Edge_map", Edge_map.astype(np.uint8)*255)
#                 cv2.imshow("High_region_Image", High_region_Image.astype(np.uint8)*255)
#                 cv2.imshow("Hand_mask_Image", Hand_mask_Image.astype(np.uint8)*255)
                cv2.imshow("tracking_image", tracking_image)
                
#                 cv2.imshow("ZED_gray", grayImage)
    
            cv2.imshow("ZED", RGB_image)
#             cv2.imshow("Depth img", depth_for_display.get_data())

            key = cv2.waitKey(1)
        else :
            key = cv2.waitKey(1)

    cam.close()


In [5]:
if __name__ == "__main__":
    main()

  Quit : CTRL+C

Inliner Number
 599733
Inliner plane
 [-4.80251194e+02 -1.34452073e+03 -9.84207105e+03  4.86689540e+06]
firstly RANSAM
pos :  (1021, 116)
Dist:  449.62395368428804
pos :  (1019, 117)
Dist:  451.29311714256954
pos :  (1023, 115)
Dist:  451.4235092362598
pos :  (1028, 113)
Dist:  457.5508147054822
pos :  (1037, 111)
Dist:  460.24716811676427
pos :  (1044, 110)
Dist:  466.74337075324314
pos :  (1051, 108)
Dist:  467.18488318869134
pos :  (1055, 106)
Dist:  468.53466513458056
pos :  (1059, 106)
Dist:  470.6846360245213
pos :  (49, 380)
Dist:  458.74338047176315
pos :  (47, 384)
Dist:  458.3985067155505
pos :  (51, 396)
Dist:  465.19595969475773
pos :  (57, 421)
Dist:  489.38410328666237
pos :  (65, 444)
Dist:  497.9589086400914
pos :  (66, 448)
Dist:  501.75892630124747
pos :  (71, 464)
Dist:  512.0783203030247
pos :  (79, 477)
Dist:  503.3178292156927
pos :  (78, 473)
Dist:  440.9161300348362
pos :  (152, 504)
Dist:  427.3977443734071
pos :  (120, 505)
Dist:  476.24280053

KeyboardInterrupt: 

In [ ]:
a = np.zeros((720,1280))
a[200:520,300:980] = 1
plt.imshow(a)